In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.2MB 59kB/s 
     |████████████████████████████████| 204kB 47.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=ed5d3092a33c9b93742bfff576cae4c21a88018cc3089d848b60b805b224eda1
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 21 not upgraded.
Need to get 35.8 MB of archives.
After this operation, 140 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [3]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [4]:
datos_patentamientos = [
    ('MHG 100', 'Fiat', 'Siena', 1, 'auto', 'Buenos Aires', '20170315'),
    ('MHG 101', 'Ford', 'Cargo 712', 2, 'camion', 'Chaco', '20170319'),
    ('MHG 102', 'Ford', 'Cargo 712', 4, 'camion', 'Buenos Aires', '20170401'),
    ('MHG 103', 'Fiat', 'Siena', 2, 'auto', 'Buenos Aires', '20170402'),
    ('MHG 104', 'Chevrolet', 'Sonic', 1, 'auto', 'Buenos Aires', '20170402'),
    ('MHG 105', 'Fiat', 'Siena', 3, 'auto', 'Uruguay', '20170403'),
    ('MHG 106', 'Fiat', 'Siena', 1, 'auto', 'Buenos Aires', '20170405'),
    ('MHG 107', 'Chevrolet', 'Sonic', 2, 'auto', 'Buenos Aires', '20170417'),
    ('MHG 108', 'Chevrolet', 'Sonic', 1, 'auto', 'Buenos Aires', '20170419'),
    ('MHG 109', 'Ford', 'Cargo 712', 4, 'camion', 'Buenos Aires', '20170419'),
    ('MHG 110', 'Ford', 'Cargo 712', 2, 'camion', 'Buenos Aires', '20170419'),
    ('MHG 111', 'Fiat', 'Siena', 3, 'auto', 'Cordoba', '20170420'),
    ('MHG 112', 'Chevrolet', 'Sonic', 2, 'auto', 'Buenos Aires', '20170421'),
    ('MHG 113', 'Fiat', 'Sedan', 2, 'auto', 'Buenos Aires', '20170423'),
    ('MHG 114', 'Fiat', 'Sedan', 1, 'auto', 'Buenos Aires', '20170424'),
    ('MHG 115', 'Honda', 'Hornet 160R', 1, 'moto', 'Buenos Aires', '20170425'),
    ('MHG 116', 'Honda', 'Hornet 160R', 1, 'moto', 'Buenos Aires', '20170425'),
    ('MHG 117', 'Ducati', 'SuperSport', 1, 'moto', 'Buenos Aires', '20170426'),
    ('MHG 118', 'Scania', '420', 4, 'camion', 'Buenos Aires', '20170426')
]

9- Se cuenta con un RDD con información sobre patentamientos de autos con la siguiente
información (patente, marca, modelo, versión, tipo_vehiculo, provincia, fecha), donde
tipo_vehiculo indica si la unidad patentada es auto, pickup, camión o moto. Se pide generar un
programa en pySpark que indique la marca y modelo del auto más patentado por tipo de
vehículo en la provincia de Buenos Aires en el mes de Abril de 2017.

In [5]:
patentamientos = sc.parallelize(datos_patentamientos)
patentamientos.count()

19

In [6]:
patentamientos_abril = patentamientos.filter(lambda x: x[6] > '20170400' and x[6] < '20170500' and x[5] == 'Buenos Aires')
patentamientos_abril.collect()

[('MHG 102', 'Ford', 'Cargo 712', 4, 'camion', 'Buenos Aires', '20170401'),
 ('MHG 103', 'Fiat', 'Siena', 2, 'auto', 'Buenos Aires', '20170402'),
 ('MHG 104', 'Chevrolet', 'Sonic', 1, 'auto', 'Buenos Aires', '20170402'),
 ('MHG 106', 'Fiat', 'Siena', 1, 'auto', 'Buenos Aires', '20170405'),
 ('MHG 107', 'Chevrolet', 'Sonic', 2, 'auto', 'Buenos Aires', '20170417'),
 ('MHG 108', 'Chevrolet', 'Sonic', 1, 'auto', 'Buenos Aires', '20170419'),
 ('MHG 109', 'Ford', 'Cargo 712', 4, 'camion', 'Buenos Aires', '20170419'),
 ('MHG 110', 'Ford', 'Cargo 712', 2, 'camion', 'Buenos Aires', '20170419'),
 ('MHG 112', 'Chevrolet', 'Sonic', 2, 'auto', 'Buenos Aires', '20170421'),
 ('MHG 113', 'Fiat', 'Sedan', 2, 'auto', 'Buenos Aires', '20170423'),
 ('MHG 114', 'Fiat', 'Sedan', 1, 'auto', 'Buenos Aires', '20170424'),
 ('MHG 115', 'Honda', 'Hornet 160R', 1, 'moto', 'Buenos Aires', '20170425'),
 ('MHG 116', 'Honda', 'Hornet 160R', 1, 'moto', 'Buenos Aires', '20170425'),
 ('MHG 117', 'Ducati', 'SuperSport', 1

In [7]:
patentamientos_por_tipo = patentamientos_abril.map(lambda x: ((x[4],x[1],x[2]),1))
patentamientos_por_tipo.collect()

[(('camion', 'Ford', 'Cargo 712'), 1),
 (('auto', 'Fiat', 'Siena'), 1),
 (('auto', 'Chevrolet', 'Sonic'), 1),
 (('auto', 'Fiat', 'Siena'), 1),
 (('auto', 'Chevrolet', 'Sonic'), 1),
 (('auto', 'Chevrolet', 'Sonic'), 1),
 (('camion', 'Ford', 'Cargo 712'), 1),
 (('camion', 'Ford', 'Cargo 712'), 1),
 (('auto', 'Chevrolet', 'Sonic'), 1),
 (('auto', 'Fiat', 'Sedan'), 1),
 (('auto', 'Fiat', 'Sedan'), 1),
 (('moto', 'Honda', 'Hornet 160R'), 1),
 (('moto', 'Honda', 'Hornet 160R'), 1),
 (('moto', 'Ducati', 'SuperSport'), 1),
 (('camion', 'Scania', '420'), 1)]

In [17]:
marcas_y_modelos_putos = patentamientos_por_tipo.reduceByKey(lambda x,y: x + y).map(lambda x: (x[0][0], (x[0][1],x[0][2],x[1]))  )
marcas_y_modelos_putos.collect()

[('camion', ('Ford', 'Cargo 712', 3)),
 ('auto', ('Fiat', 'Siena', 2)),
 ('auto', ('Chevrolet', 'Sonic', 4)),
 ('moto', ('Honda', 'Hornet 160R', 2)),
 ('moto', ('Ducati', 'SuperSport', 1)),
 ('camion', ('Scania', '420', 1)),
 ('auto', ('Fiat', 'Sedan', 2))]

In [18]:
marcas_y_modelos_putos.reduceByKey(lambda x,y: x if x[2] > y[2] else y).collect()

[('auto', ('Chevrolet', 'Sonic', 4)),
 ('camion', ('Ford', 'Cargo 712', 3)),
 ('moto', ('Honda', 'Hornet 160R', 2))]

auto: Chevrolet Sonic

moto: Honda Hornet 160R

camion: Ford Cargo 712